In [ ]:
import os
os.chdir("/content/drive/MyDrive/phd/datasets")

In [ ]:
os.listdir()

In [ ]:
!tar -xf data_medulloblastoma.tar

In [ ]:
os.chdir("data_medulloblastoma/")
os.listdir()

In [ ]:
import pandas as pd

In [ ]:
df_files = pd.read_csv("res_group_snf.txt", sep="\t", index_col=1).drop("ID", axis=1)
df_files.head(2)

In [ ]:
for file, sep in zip(['matrix_rnaseq_log2_rpkm.txt', 'matrix_mirna_log2_cpm.txt', 'matrix_proteome_log2ratio.txt'], ["\t", " ", "\t"]):
    df = pd.read_csv(file, sep = sep)
    print(df.shape)
    print(df.head(2))

In [ ]:
df = pd.read_csv('matrix_rnaseq_log2_rpkm.txt', sep="\t", index_col=0)
df.head(2)

In [ ]:
df_files.columns

In [ ]:
df_files["group_snf"].unique()

In [ ]:
!python3 -m pip install scanpy > /dev/null &2>1
import scanpy as sc

In [ ]:
adata = sc.AnnData(X = df.transpose(),
                   obs = df_files.reindex(index=df.columns))

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=5000)

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
hvg = adata.var[adata.var["highly_variable"]==True].index

In [ ]:
df_mirna = pd.read_csv("matrix_mirna_log2_cpm.txt", sep=" ")
df_mirna.head(2)

In [ ]:
df = df.reindex(index=hvg).append(df_mirna).dropna(how="any", axis=1, thresh=2000)
print(df.shape)
df.head(2)

# Model

In [ ]:
!echo "deb https://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
!apt-key adv --keyserver keys.openpgp.org --recv-key 612DEFB798507F25 > /dev/null &2>1
!apt-get update > /dev/null &2>1 && apt-get install python3-graph-tool --yes > /dev/null &2>1
!cd ../../ && git clone https://github.com/fvalle1/trisbm.git

In [ ]:
import sys
sys.path.append("/content/drive/My Drive/phd/trisbm")

In [ ]:
from trisbm.trisbm import trisbm

In [ ]:
model = trisbm()

In [ ]:
model.make_graph(df.applymap(lambda rpkm: 2**rpkm -1 ), get_kind=lambda g: 2 if "hsa" in g else 1)

In [ ]:
model.fit(n_init=3, verbose=True)
!rm -rf trisbm
!mkdir -p trisbm
os.chdir("trisbm")
model.save_data()
os.chdir("../")
!tar -cf trisbm.tar trisbm/

In [ ]:
df_files.to_csv("files.dat")